In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import mne
from mne.preprocessing import ICA

from Cho2017 import SubjectData


plt.rcParams['figure.figsize'] = [12, 8]

# # Make sure ipympl is installed (via pip) for interactive plots in Jupyter 
# %matplotlib widget

In [ ]:
subject = SubjectData('../Data/mat_data/s01.mat')

In [ ]:
rej = {'eeg': 150e-6}

im_left_epochs = subject.get_epochs('imagery_left', tmin=-0.5, tmax=2, picks=['eeg'], reject_criteria=rej, verbose=False)
im_right_epochs = subject.get_epochs('imagery_right', tmin=-0.5, tmax=2, picks=['eeg'], reject_criteria=rej, verbose=False)

In [ ]:
n_ica_comps = 64
ica = mne.preprocessing.ICA(n_components=n_ica_comps, random_state=97, max_iter=1000)
ica.fit(im_left_epochs)
# ica.plot_sources(im_left_epochs);
ica.plot_components();
# ica.plot_properties(im_left_epochs);

In [ ]:
ica.plot_properties(im_left_epochs);

In [ ]:
left_keep_comps = list(range(63))
ignore_comps = ignore_comps = list(filter(lambda i: i not in left_keep_comps, list(range(n_ica_comps))))
denoised_im_left_epochs = im_left_epochs.copy()
ica.apply(denoised_im_left_epochs, exclude=ignore_comps)

In [ ]:
# ica = mne.preprocessing.ICA(n_components=n_ica_comps, random_state=97, max_iter=1000)
# ica.fit(im_right_epochs)
# ica.plot_components();

In [ ]:
# right_keep_comps = [0, 1, 2, 4]
right_keep_comps = left_keep_comps
# ignore_comps = list(filter(lambda i: i not in right_keep_comps, list(range(n_ica_comps))))
denoised_im_right_epochs = im_right_epochs.copy()
ica.apply(denoised_im_right_epochs, exclude=ignore_comps)

In [ ]:
from sklearn import datasets
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import ShuffleSplit, cross_val_score
from sklearn.pipeline import Pipeline

from mne.decoding import CSP

In [ ]:
assert len(left_keep_comps) == len(right_keep_comps)

data = np.vstack((denoised_im_left_epochs.get_data(), denoised_im_right_epochs.get_data()))
# # Alternatively can try with ICA un-mixed sources rather than the denoised signal
source_left = ica.get_sources(im_left_epochs, start=0.5, stop=2.5)
source_right = ica.get_sources(im_right_epochs, start=0.5, stop=2.5)
data = np.vstack((source_right.get_data()[:, left_keep_comps, :], source_left.get_data()[:, right_keep_comps, :]))

labels = np.hstack((im_left_epochs.events[:, -1] - 1, im_right_epochs.events[:, -1]))  # Set left events to 0 to have 2 distinct class labels (left: 0 and right: 1)

In [ ]:
data.shape

In [ ]:
scores = []
cv = ShuffleSplit(10, test_size=0.25, random_state=42)
cv_split = cv.split(data)

In [ ]:
csp = CSP(n_components=2, reg=None, log=False, norm_trace=False)   # Cho 2017 uses 2 components
lda = LinearDiscriminantAnalysis()

clf = Pipeline([('CSP', csp), ('LDA', lda)])
scores = cross_val_score(clf, data, labels, cv=cv, n_jobs=8, error_score='raise')

In [ ]:
class_balance = np.mean(labels == labels[0])
class_balance = max(class_balance, 1. - class_balance)
print("Classification accuracy: %f / Chance level: %f" % (np.mean(scores),
                                                          class_balance))

In [ ]:
print(data.shape)
print(labels.shape, labels)
print(scores)

In [ ]:
plt.imshow(data[3], aspect='auto')

In [ ]:
plt.imshow(data[-3], aspect='auto')

# Left/Right Joint Denoising

In [ ]:
joint_im_epochs = mne.concatenate_epochs([im_left_epochs.copy(), im_right_epochs.copy()], add_offset=True)
labels = np.hstack((im_left_epochs.events[:, -1] - 1, im_right_epochs.events[:, -1]))  # Set left events to 0 to have 2 distinct class labels (left: 0 and right: 1)
ica = mne.preprocessing.ICA(n_components=n_ica_comps, random_state=97, max_iter=1000)
ica.fit(joint_im_epochs)
ica.plot_components();

In [ ]:
joint_keep_comps = [0, 2]
ignore_comps = list(filter(lambda i: i not in joint_keep_comps, list(range(n_ica_comps))))
denoised_joint_im_epochs = joint_im_epochs.copy()
ica.apply(denoised_joint_im_epochs, exclude=ignore_comps)

In [ ]:
data = ica.get_sources(denoised_joint_im_epochs, start=0.5, stop=2.5).get_data()
# data = denoised_joint_im_epochs.get_data()

In [ ]:
scores = []
cv = ShuffleSplit(10, test_size=0.25, random_state=42)
cv_split = cv.split(data)

csp = CSP(n_components=2, reg=None, log=False, norm_trace=False)   # Cho 2017 uses 2 components
lda = LinearDiscriminantAnalysis()

clf = Pipeline([('CSP', csp), ('LDA', lda)])
scores = cross_val_score(clf, data, labels, cv=cv, n_jobs=8, error_score='raise')

class_balance = np.mean(labels == labels[0])
class_balance = max(class_balance, 1. - class_balance)
print("Classification accuracy: %f / Chance level: %f" % (np.mean(scores),
                                                          class_balance))

In [ ]:
print(data.shape)
print(labels.shape, labels)
print(scores)

In [ ]:
plt.imshow(data[3], aspect='auto')

In [ ]:
plt.imshow(data[-3], aspect='auto')

## Applying Corrmap from noise trial and the MI trials to find bad components

## blinking

In [ ]:
picks = ['FP1', 'FPZ', 'FP2', 'F3', 'F4'] # None / ['eeg'] for first 20 channels or list of channel names for those specified channels

# Since blinking is performed in 2x 5s trials, it's possible the discontinuity at the trial boundary may be falsely marked as an EOG
blinking = [subject.raw_noise['blinking'].copy().crop(tmin=0, tmax=5), subject.raw_noise['blinking'].copy().crop(tmin=5, tmax=None)]
blinking[0].copy().pick(picks).plot();
blinking[1].copy().pick(picks).plot();

from mne import concatenate_epochs
from mne.preprocessing import find_eog_events
find_eog_ch = 'FP1'
tmin = -0.25
tmax = 0.25
baseline_toffset = 0.01

trial0_blink_events = find_eog_events(blinking[0], ch_name=find_eog_ch, filter_length=2561)    # Match filter length to length of signal
trial1_blink_events = find_eog_events(blinking[1], ch_name=find_eog_ch, filter_length=2560)

trial0_blink_epochs = mne.Epochs(blinking[0], trial0_blink_events, tmin=tmin, tmax=tmax, baseline=(tmin, tmin+baseline_toffset), reject=None, preload=True)
trial1_blink_epochs = mne.Epochs(blinking[1], trial1_blink_events, tmin=tmin, tmax=tmax, baseline=(tmin, tmin+baseline_toffset), reject=None, preload=True)
# trial0_blink_epochs.plot(picks=picks);
# trial1_blink_epochs.plot(picks=picks);

blink_epochs = concatenate_epochs([trial0_blink_epochs, trial1_blink_epochs])
blink_epochs.plot(picks=picks);

ica_blink = mne.preprocessing.ICA(n_components=n_ica_comps, random_state=97, max_iter=1000)
ica_blink.fit(blink_epochs)

In [ ]:
from mne.preprocessing import corrmap

ica_excludes = [15, 23, 24, 27, 28, 35, 36, 37, 39, 41, 42, 43 , 45, 46, 48, 50, 51, 54, 55, 56, 57, 58, 59, 60, 61, 63]

for i in range(63):
    if i not in ica_excludes:
        corrmap([ica, ica_blink], (1, i), label='test');

In [ ]:
ica.labels_['test']

### eye_up-down

In [ ]:
picks = ['FP1', 'FPZ', 'FP2', 'F3', 'F4'] # None / ['eeg'] for first 20 channels or list of channel names for those specified channels

# Since blinking is performed in 2x 5s trials, it's possible the discontinuity at the trial boundary may be falsely marked as an EOG
eye_up_down = [subject.raw_noise['eye_up-down'].copy().crop(tmin=0, tmax=5), subject.raw_noise['eye_up-down'].copy().crop(tmin=5, tmax=None)]
eye_up_down[0].copy().pick(picks).plot();
eye_up_down[1].copy().pick(picks).plot();

from mne import concatenate_epochs
from mne.preprocessing import find_eog_events
find_eog_ch = 'FP1'
tmin = -0.25
tmax = 0.25
baseline_toffset = 0.01

trial0_eye_up_down_events = find_eog_events(eye_up_down[0], ch_name=find_eog_ch, filter_length=2561)    # Match filter length to length of signal
trial1_eye_up_down_events = find_eog_events(eye_up_down[1], ch_name=find_eog_ch, filter_length=2560)

trial0_eye_up_down_epochs = mne.Epochs(eye_up_down[0], trial0_eye_up_down_events, tmin=tmin, tmax=tmax, baseline=(tmin, tmin+baseline_toffset), reject=None, preload=True)
trial1_eye_up_down_epochs = mne.Epochs(eye_up_down[1], trial1_eye_up_down_events, tmin=tmin, tmax=tmax, baseline=(tmin, tmin+baseline_toffset), reject=None, preload=True)
# trial0_eye_up_down_epochs.plot(picks=picks);

eye_up_down_epochs = concatenate_epochs([trial0_eye_up_down_epochs, trial1_eye_up_down_epochs])
eye_up_down_epochs.plot(picks=picks);

from mne.preprocessing import ICA

n_ica_comps = 64
ica_eye_up_down = mne.preprocessing.ICA(n_components=n_ica_comps, random_state=97, max_iter=1000)
ica_eye_up_down.fit(eye_up_down_epochs)
# ica_eye_up_down.plot_sources(eye_up_down_epochs);
# ica_eye_up_down.plot_components();

In [ ]:
from mne.preprocessing import corrmap

# eye_up_down
ica_eye_up_down_excludes = [16, 23, 26, 27, 28, 29, 30, 31, 37, 41, 42, 44, 47, 48, 51, 52, 54, 55, 57, 58, 59, 60, 61,62, 63]

for i in range(63):
    if i not in ica_eye_up_down_excludes:
        corrmap([ica, ica_eye_up_down], (1, i), label='test_eye_up_down');

In [ ]:
ica.labels_['test_eye_up_down']

### eye_left-right

In [ ]:
picks = ['FP1', 'FPZ', 'FP2', 'F3', 'F4'] # None / ['eeg'] for first 20 channels or list of channel names for those specified channels

# Since blinking is performed in 2x 5s trials, it's possible the discontinuity at the trial boundary may be falsely marked as an EOG
eye_left_right = [subject.raw_noise['eye_left-right'].copy().crop(tmin=0, tmax=5), subject.raw_noise['eye_left-right'].copy().crop(tmin=5, tmax=None)]
eye_left_right[0].copy().pick(picks).plot();
eye_left_right[1].copy().pick(picks).plot();

from mne import concatenate_epochs
from mne.preprocessing import find_eog_events
find_eog_ch = 'FP1'
tmin = -0.25
tmax = 0.25
baseline_toffset = 0.01

trial0_eye_left_right_events = find_eog_events(eye_left_right[0], ch_name=find_eog_ch, filter_length=2561)    # Match filter length to length of signal
trial1_eye_left_right_events = find_eog_events(eye_left_right[1], ch_name=find_eog_ch, filter_length=2560)

trial0_eye_left_right_epochs = mne.Epochs(eye_left_right[0], trial0_eye_left_right_events, tmin=tmin, tmax=tmax, baseline=(tmin, tmin+baseline_toffset), reject=None, preload=True)
trial1_eye_left_right_epochs = mne.Epochs(eye_left_right[1], trial1_eye_left_right_events, tmin=tmin, tmax=tmax, baseline=(tmin, tmin+baseline_toffset), reject=None, preload=True)
# trial0_eye_up_down_epochs.plot(picks=picks);

eye_left_right_epochs = concatenate_epochs([trial0_eye_left_right_epochs, trial1_eye_left_right_epochs])
eye_left_right_epochs.plot(picks=picks);

from mne.preprocessing import ICA

n_ica_comps = 64
ica_eye_left_right = mne.preprocessing.ICA(n_components=n_ica_comps, random_state=97, max_iter=1000)
ica_eye_left_right.fit(eye_left_right_epochs)
# ica_eye_left_right.plot_sources(eye_left_right_epochs);
# ica_eye_left_right.plot_components();

In [ ]:
from mne.preprocessing import corrmap
# eye_left_right
ica_eye_left_right_excludes = [20, 23, 27, 28, 32, 33, 34, 36, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63]

for i in range(63):
    if i not in ica_eye_left_right_excludes:
        corrmap([ica, ica_eye_left_right], (1, i), label='test_eye_left_right');

In [ ]:
ica.labels_['test_eye_left_right']

### jaw

In [ ]:
picks = ['FP1', 'FPZ', 'FP2', 'F3', 'F4'] # None / ['eeg'] for first 20 channels or list of channel names for those specified channels

# Since blinking is performed in 2x 5s trials, it's possible the discontinuity at the trial boundary may be falsely marked as an EOG
jaw = [subject.raw_noise['jaw'].copy().crop(tmin=0, tmax=5), subject.raw_noise['jaw'].copy().crop(tmin=5, tmax=None)]
jaw[0].copy().pick(picks).plot();
jaw[1].copy().pick(picks).plot();

from mne import concatenate_epochs
from mne.preprocessing import find_eog_events
find_eog_ch = 'FP1'
tmin = -0.25
tmax = 0.25
baseline_toffset = 0.01

trial0_jaw_events = find_eog_events(jaw[0], ch_name=find_eog_ch, filter_length=2561)    # Match filter length to length of signal
trial1_jaw_events = find_eog_events(jaw[1], ch_name=find_eog_ch, filter_length=2560)

trial0_jaw_epochs = mne.Epochs(jaw[0], trial0_jaw_events, tmin=tmin, tmax=tmax, baseline=(tmin, tmin+baseline_toffset), reject=None, preload=True)
trial1_jaw_epochs = mne.Epochs(jaw[1], trial1_jaw_events, tmin=tmin, tmax=tmax, baseline=(tmin, tmin+baseline_toffset), reject=None, preload=True)
# trial0_eye_up_down_epochs.plot(picks=picks);

jaw_epochs = concatenate_epochs([trial0_jaw_epochs, trial1_jaw_epochs])
jaw_epochs.plot(picks=picks);

from mne.preprocessing import ICA

n_ica_comps = 64
ica_jaw = mne.preprocessing.ICA(n_components=n_ica_comps, random_state=97, max_iter=1000)
ica_jaw.fit(jaw_epochs)
# ica_jaw.plot_sources(jaw_epochs);
# ica_jaw.plot_components();

In [ ]:
from mne.preprocessing import corrmap
# jaw
ica_jaw_excludes = [5, 6, 7, 11, 43, 44, 48, 49, 50, 55, 56, 57, 58, 59, 61, 62, 63]

for i in range(63):
    if i not in ica_jaw_excludes:
        corrmap([ica, ica_jaw], (1, i), label='test_jaw');

In [ ]:
ica.labels_['test_jaw']

### head_left-right

In [ ]:
picks = ['FP1', 'FPZ', 'FP2', 'F3', 'F4'] # None / ['eeg'] for first 20 channels or list of channel names for those specified channels

# Since blinking is performed in 2x 5s trials, it's possible the discontinuity at the trial boundary may be falsely marked as an EOG
head_left_right = [subject.raw_noise['head_left-right'].copy().crop(tmin=0, tmax=5), subject.raw_noise['head_left-right'].copy().crop(tmin=5, tmax=None)]
head_left_right[0].copy().pick(picks).plot();
head_left_right[1].copy().pick(picks).plot();

from mne import concatenate_epochs
from mne.preprocessing import find_eog_events
find_eog_ch = 'FP1'
tmin = -0.25
tmax = 0.25
baseline_toffset = 0.01

trial0_head_left_right_events = find_eog_events(head_left_right[0], ch_name=find_eog_ch, filter_length=2561)    # Match filter length to length of signal
trial1_head_left_right_events = find_eog_events(head_left_right[1], ch_name=find_eog_ch, filter_length=2560)

trial0_head_left_right_epochs = mne.Epochs(head_left_right[0], trial0_head_left_right_events, tmin=tmin, tmax=tmax, baseline=(tmin, tmin+baseline_toffset), reject=None, preload=True)
trial1_head_left_right_epochs = mne.Epochs(head_left_right[1], trial1_head_left_right_events, tmin=tmin, tmax=tmax, baseline=(tmin, tmin+baseline_toffset), reject=None, preload=True)
# trial0_eye_up_down_epochs.plot(picks=picks);

head_left_right_epochs = concatenate_epochs([trial0_head_left_right_epochs, trial1_head_left_right_epochs])
head_left_right_epochs.plot(picks=picks);

from mne.preprocessing import ICA

n_ica_comps = 64
ica_head_left_right = mne.preprocessing.ICA(n_components=n_ica_comps, random_state=97, max_iter=1000)
ica_head_left_right.fit(head_left_right_epochs)
# ica_head_left_right.plot_sources(head_left_right_epochs);
# ica_head_left_right.plot_components();

In [ ]:
from mne.preprocessing import corrmap
# head_left_right
ica_head_left_right_excludes = [6, 7, 10, 11, 13, 15, 17, 18, 21, 23, 26, 29, 32, 33, 39, 42, 45, 48, 49, 53, 54, 57, 58, 60, 61, 62, 63]

for i in range(63):
    if i not in ica_head_left_right_excludes:
        corrmap([ica, ica_head_left_right], (1, i), label='test_head_left_right');

In [ ]:
ica.labels_['test_head_left_right']